# 버블 차트 시각화

In [1]:
# 라이브러리 모음
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import plotly.graph_objects as go
import plotly.express as px

# 모듈 모음
import modules as mds

In [ ]:
# Windows에서 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # '맑은 고딕'이 설치되어 있을 경우
plt.rcParams['axes.unicode_minus'] = False     # 마이너스(-) 부호 깨짐 방지

In [2]:
# macOS에서 한글 폰트 설정
plt.rcParams['font.family'] = 'AppleGothic'   # macOS 기본 한글 폰트
plt.rcParams['axes.unicode_minus'] = False

In [3]:
# 이용률 / 일사량 버블 차트 시각화

df = pd.read_excel("dataset/최종일사량합친데이터.xlsx")
df_2024 = df[df["연도"] == 2024].copy()

# 버블 크기 (설비용량 기반, 최소값 보정)
df_2024["설비용량"] = df_2024["설비용량(MW)"].apply(lambda v: max(50, float(v)))

# 숫자형 변환
x = pd.to_numeric(df_2024["일사량(MJ/m2)"], errors="coerce")
y = pd.to_numeric(df_2024["설비이용률(%)"], errors="coerce")

xmin, xmax = float(x.min()), float(x.max())
ymin, ymax = float(y.min()), float(y.max())
padx = (xmax - xmin) * 0.05
pady = (ymax - ymin) * 0.05

mean_x = x.mean()
mean_y = y.mean()

# color map 설정
color_map = {
    "서울": "#E74C3C",
    "부산": "#3498DB",
    "대구": "#2ECC71",
    "인천": "#F39C12",
    "광주": "#9B59B6",
    "대전": "#1ABC9C",
    "울산": "#E67E22",
    "경기": "#34495E",
    "강원": "#F1C40F",
    "경북": "#E91E63",
    "경남": "#00BCD4",
    "전북": "#FF9800",
    "전남": "#795548",
    "충북": "#607D8B",
    "충남": "#4CAF50",
    "제주": "#673AB7"
}

# 산점도
fig = px.scatter(
    df_2024,
    x="일사량(MJ/m2)",
    y="설비이용률(%)",
    size="설비용량",
    color="지역",
    hover_name="지역",
    text="지역",
    size_max=60,
    title="2024년 지역별 일사량과 설비이용률 비교 분석",
    color_discrete_map=color_map
)

# 기본 텍스트 제거 (조건부로 다시 추가)
fig.update_traces(text=None)

# 큰/작은 버블 라벨 처리
THRESH = 500  # 작은 버블 기준 (MW)

for _, row in df_2024.iterrows():
    if row["설비용량"] >= THRESH:
        # 큰 버블 → 중앙에 흰 글자(굵게 보이게 HTML 태그 사용)
        fig.add_annotation(
            x=row["일사량(MJ/m2)"],
            y=row["설비이용률(%)"],
            text=f"<b>{row['지역']}</b>",
            showarrow=False,
            font=dict(color="white", size=11)
        )
    else:
        # 작은 버블 → 바깥쪽 라벨 + 화살표
        fig.add_annotation(
            x=row["일사량(MJ/m2)"],
            y=row["설비이용률(%)"],
            text=row["지역"],
            showarrow=True, arrowhead=2,
            ax=15, ay=-15,
            font=dict(color="black", size=11),
            bgcolor="rgba(255,255,255,0.6)"
        )

# 크기 설정
fig.update_layout(width=1300,height=800)

# 평균선
fig.add_shape(type="line", x0=mean_x, y0=ymin, x1=mean_x, y1=ymax,
                line=dict(color="red", dash="dash", width=2))
fig.add_shape(type="line", x0=xmin, y0=mean_y, x1=xmax, y1=mean_y,
                line=dict(color="red", dash="dash", width=2))

# 사분면 라벨
fig.add_annotation(x=xmax, y=ymax, text="성과 우수 (자원↑, 효율↑)",
                    showarrow=False, font=dict(color="green"), xanchor="right", yanchor="bottom")
fig.add_annotation(x=xmin, y=ymax, text="효율 우수 (자원↓, 효율↑)",
                    showarrow=False, font=dict(color="blue"), xanchor="left", yanchor="bottom")
fig.add_annotation(x=xmax, y=ymin, text="잠재력 有 (자원↑, 효율↓)",
                    showarrow=False, font=dict(color="orange"), xanchor="right", yanchor="top")
fig.add_annotation(x=xmin, y=ymin, text="저효율 (자원↓, 효율↓)",
                    showarrow=False, font=dict(color="grey"), xanchor="left", yanchor="top")

# 축 범위
fig.update_xaxes(range=[xmin - padx, xmax + padx], title="합계 일사량 (MJ/m²)")
fig.update_yaxes(range=[ymin - pady, ymax + pady + 0.2], title="설비이용률 (%)")
fig.update_layout(template="plotly_white")

fig.show()